In [1]:
import torch
from torch_geometric.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from gnn import GNN
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
import argparse
import time
import numpy as np

/common/home/pv217/.conda/envs/gln_new/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from ogb.graphproppred import PygGraphPropPredDataset, Evaluator
torch.autograd.detect_anomaly()
cls_criterion = torch.nn.BCEWithLogitsLoss()
reg_criterion = torch.nn.MSELoss()


/tmp/ipykernel_1227944/775782116.py:2: UserWarning: Anomaly Detection has been enabled. This mode will increase the runtime and should only be enabled for debugging.
  torch.autograd.detect_anomaly()


In [3]:
class args:
    device = 2
    gnn = 'gcn'
    drop_ratio = 0.5
    logical_reg = 0.15
    num_layer_gnn = 5
    emb_dim = 300
    batch_size = 128
    epochs=100
    num_workers =0
    dataset = 'ogbg-molpcba'
    graph_pooling='mean'
    feature = 'full'
    filename = 'GLN_model'
    
    
    
    
    

In [4]:
def train(model, device, loader, optimizer, task_type,logical_reg):
    model.train()
#    print([i for i in model.named_parameters()])
    for step, batch in enumerate(tqdm(loader, desc="Iteration")):
        batch = batch.to(device)
        # if batch.x.shape[0] == 1 or batch.batch[-1] == 0:
        #     pass
        # else:
        constraints,pred = model(batch,1)
        #print(constraints)
        optimizer.zero_grad()
        ## ignore nan targets (unlabeled) when computing training loss.
        y = batch.y[:,:1]
        pred = pred[:,:1]
        if model.graph_pooling == 'logical':
            
            false_vector = model.logic_not(model.true_vector)  # we compute the representation
            
             #         here, we maximize the similarity between not true and false
            r_not_true = (1 - F.cosine_similarity(model.logic_not(model.true_vector), false_vector, dim=0))
            
            # here, we maximize the similarity between not not x and x
            r_not_not_self = (1 - F.cosine_similarity(model.logic_not(model.logic_not(constraints)), constraints)).mean()

            # here, we minimize the similarity between not x and x
            r_not_self = (1 + F.cosine_similarity(model.logic_not(constraints), constraints)).mean()

            # here, we maximize the similarity between not not true and true
            r_not_not_true = (1 - F.cosine_similarity(
                model.logic_not(model.logic_not(model.true_vector)), model.true_vector,dim=0))

   
            # And

    
            # here, we maximize the similarity between x AND True and x
            r_and_true = (1 - F.cosine_similarity(
                model.logic_and(constraints, model.true_vector.expand_as(constraints)), constraints)).mean()

            # here, we maximize the similarity between x AND False and False
            r_and_false = (1 - F.cosine_similarity(
                model.logic_and(constraints, false_vector.expand_as(constraints)),
                false_vector.expand_as(constraints))).mean()

            # here, we maximize the similarity between x AND x and x
            r_and_self = (1 - F.cosine_similarity(model.logic_and(constraints, constraints), constraints)).mean()

            # here, we maximize the similarity between x AND not x and False
            r_and_not_self = (1 - F.cosine_similarity(
                model.logic_and(constraints, model.logic_not(constraints)),
                false_vector.expand_as(constraints))).mean()

            # same rule as before, but we flipped operands
            r_and_not_self_inverse = (1 - F.cosine_similarity(
                model.logic_and(model.logic_not(constraints), constraints),
                false_vector.expand_as(constraints))).mean()
            
            

            # True/False rule
            # here, we minimize the similarity between True and False
            true_false = 1 + F.cosine_similarity(model.true_vector, false_vector, dim=0)

#             # here, we maximize similatrity between True AND True and True | R3/5 True
            true_and_true = (1 - F.cosine_similarity(model.logic_and(model.true_vector,model.true_vector,dim=0),model.true_vector, dim=0))
            # here, we maximize similatrity between True AND False and False AND True 
            true_and_false_false_and_false = (1 -F.cosine_similarity(
model.logic_and(false_vector,model.true_vector,dim=0),model.logic_and(model.true_vector,false_vector,dim=0), dim=0))
            
            
#             print(r_not_not_self,r_not_self,r_not_not_not)
            
            r_loss = (r_not_true + 
            r_not_not_self + r_not_self + r_not_not_true +
                  r_and_true + r_and_false + r_and_self + r_and_not_self + r_and_not_self_inverse 
#                       + true_and_true  
#                       + true_false
#                       + true_and_false_false_and_false
                     )
            #print(r_loss)
            
            bs = y.shape[0]
#             print(y.shape)
            is_labeled = y == y
            is_labeled = is_labeled.reshape(1,bs)
            bce = torch.nn.BCELoss(reduction='sum')(pred.to(torch.float64).reshape(1,bs)[is_labeled], y.to(torch.float64).reshape(1,bs)[is_labeled])
#            print(bce)
            loss = r_loss*logical_reg + bce
            #  cls_criterion(pred.to(torch.float32)[is_labeled], batch.y.to(torch.float32)[is_labeled])
            #print(f'reg loss is {r_loss}')
        else:
            bs = y.shape[0]
#             print(y.shape)
            is_labeled = y == y
            is_labeled = is_labeled.reshape(bs,1)
#             print(y.shape,pred.shape)
            loss = cls_criterion(pred[is_labeled].to(torch.float32), y[is_labeled].to(torch.float32))
#        print(loss)
        loss.backward()
        optimizer.step()


In [5]:
def eval(model, device, loader, evaluator):
    model.eval()
    y_true = []
    y_pred = []

    for step, batch in enumerate(tqdm(loader, desc="Iteration")):
        batch = batch.to(device)

        if batch.x.shape[0] == 1:
            pass
        else:
            with torch.no_grad():
                _,pred = model(batch,0)
                
            pred = pred[:,:1]
            y = batch.y[:,:1]
            is_labeled = y == y
            is_labeled = is_labeled.view(pred.shape)
#             print(is_labeled.shape,pred.shape)
            pred = pred[is_labeled]
            y_true.append(y[is_labeled].view(pred.shape).detach().cpu())
            y_pred.append(pred.detach().cpu())
    
    y_true = torch.cat(y_true, dim = 0).numpy()
    y_pred = torch.cat(y_pred, dim = 0).numpy()

    input_dict = {"y_true": y_true, "y_pred": y_pred}
    #print(y_true)
    #print(y_pred)
    #print(y_true.shape,y_pred.shape)
    return  roc_auc_score(y_true,y_pred)

In [6]:
device = torch.device("cuda:" + str(args.device)) if torch.cuda.is_available() else torch.device("cpu")

In [7]:
dataset = PygGraphPropPredDataset(name = args.dataset)


In [8]:
graph_pooling = args.graph_pooling
if args.feature == 'full':
    pass 
elif args.feature == 'simple':
    print('using simple feature')
    # only retain the top two node/edge features
    dataset.data.x = dataset.data.x[:,:2]
    dataset.data.edge_attr = dataset.data.edge_attr[:,:2]



In [9]:
split_idx = dataset.get_idx_split()

### automatic evaluator. takes dataset name as input
evaluator = Evaluator(args.dataset)



In [10]:
train_loader = DataLoader(dataset[split_idx["train"]], batch_size=args.batch_size, shuffle=True, num_workers = args.num_workers)
valid_loader = DataLoader(dataset[split_idx["valid"]], batch_size=args.batch_size, shuffle=False, num_workers = args.num_workers)
test_loader = DataLoader(dataset[split_idx["test"]], batch_size=args.batch_size, shuffle=False, num_workers = args.num_workers)


/common/home/pv217/.conda/envs/gln_new/lib/python3.10/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [11]:
if args.gnn == 'gin':
    model = GNN(gnn_type = 'gin', num_tasks = dataset.num_tasks, device = device, num_layer = args.num_layer_gnn, emb_dim = args.emb_dim, drop_ratio = args.drop_ratio, graph_pooling=graph_pooling ,virtual_node = False).to(device)
elif args.gnn == 'gin-virtual':
    model = GNN(gnn_type = 'gin', num_tasks = dataset.num_tasks, device = device, num_layer = args.num_layer_gnn, emb_dim = args.emb_dim, drop_ratio = args.drop_ratio, graph_pooling=graph_pooling ,virtual_node = True).to(device)
elif args.gnn == 'gcn':
    model = GNN(gnn_type = 'gcn', num_tasks = dataset.num_tasks, device = device, num_layer = args.num_layer_gnn, emb_dim = args.emb_dim, drop_ratio = args.drop_ratio, graph_pooling=graph_pooling ,virtual_node = False).to(device)
elif args.gnn == 'gcn-virtual':
    model = GNN(gnn_type = 'gcn', num_tasks = dataset.num_tasks, device = device, num_layer = args.num_layer_gnn, emb_dim = args.emb_dim, drop_ratio = args.drop_ratio, graph_pooling=graph_pooling, virtual_node = True).to(device)
else:
    raise ValueError('Invalid GNN type')


In [12]:
my_list = [
               'and_layer_2.bias', 'and_layer_2.weight',
               'and_layer_1.bias', 'and_layer_1.weight',
               'not_layer_2.bias', 'not_layer_2.weight', 
               'not_layer_1.bias', 'not_layer_1.weight'
#        ,'and_layer_3.bias', 'and_layer_3.weight'
#        ,'and_layer_4.bias', 'and_layer_4.weight'
#        ,'not_layer_3.bias', 'not_layer_3.weight'
#        ,'not_layer_4.bias', 'not_layer_4.weight'
              ]

In [13]:
logical_params = (i[1] for i in list(filter(lambda kv: kv[0] in my_list, model.named_parameters())))
params = (i[1] for i in list(filter(lambda kv: kv[0] not in my_list, model.named_parameters())))
#    print(logical_params[0][1])
if graph_pooling == 'logical':
    optimizer = optim.Adam([{'params': params},
            {'params': logical_params, 'lr': 1e-5}
        ], lr=1e-3)
else:
    optimizer = optim.Adam(params, lr=1e-3)


In [14]:
valid_curve = []
test_curve = []
train_curve = []

In [15]:
for epoch in range(1, args.epochs + 1):
    print("=====Epoch {}".format(epoch))
    print('Training...')
    train(model, device, train_loader, optimizer, dataset.task_type,args.logical_reg)

    print('Evaluating...')
    train_perf = eval(model, device, train_loader, evaluator)
    valid_perf = eval(model, device, valid_loader, evaluator)
    test_perf = eval(model, device, test_loader, evaluator)

    print({'Train': train_perf, 'Validation': valid_perf, 'Test': test_perf})


=====Epoch 1
Training...


Iteration: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2738/2738 [01:34<00:00, 28.96it/s]


Evaluating...


Iteration: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 343/343 [00:05<00:00, 62.74it/s]


{'Train': 0.6949011114182343, 'Validation': 0.6825252288284802, 'Test': 0.6706881458166055}
=====Epoch 2
Training...


...

{'Train': 0.8558247339791307, 'Validation': 0.7992517222363044, 'Test': 0.7844901786377422}
=====Epoch 100
Training...


Iteration: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2738/2738 [01:42<00:00, 26.62it/s]


Evaluating...


Iteration: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 343/343 [00:04<00:00, 75.85it/s]

{'Train': 0.8561360560985964, 'Validation': 0.7958751074548077, 'Test': 0.7821615931991912}


In [16]:
valid_perf = eval(model, device, valid_loader, evaluator)

Iteration: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 343/343 [00:04<00:00, 76.11it/s]


In [31]:
valid_perf

0.7461838318313546

In [33]:
test_perf = eval(model, device, test_loader, evaluator)

Iteration: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 343/343 [00:47<00:00,  7.20it/s]


In [34]:
test_perf

0.7290123037350182

In [1]:
model.gnn_node

NameError: name 'model' is not defined